In [2]:
import requests
import pandas as pd
import networkx as nx

In [3]:
# Update the tasks here from GNPS

FBMN_TASK = "bb6766d6f48c4cb6bd84fd620da9cae5"
SIRIUS_TASK = "72542358ffca4eeda6a4b50d06b91e60"

In [32]:
# Saving FBMN Network
r = requests.get("http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=gnps_molecular_network_graphml/".format(FBMN_TASK))
with open('fbmn_network.graphml', 'wb') as f:
    f.write(r.content)

In [33]:
# Saving Sirius Formulas
r = requests.get("http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=merged_output/formula_identifications.tsv".format(SIRIUS_TASK))
with open('formula_classifications.tsv', 'wb') as f:
    f.write(r.content)
sirius_df = pd.read_csv("formula_classifications.tsv", sep="\t")

In [34]:
# Saving Sirius Structure
r = requests.get("http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=merged_output/compound_identifications.tsv".format(SIRIUS_TASK))
with open('compound_classifications.tsv', 'wb') as f:
    f.write(r.content)
fingerid_df = pd.read_csv("compound_classifications.tsv", sep="\t")

In [35]:
# Saving Sirius Canopus Classifications
r = requests.get("http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=merged_output/canopus_classification.tsv".format(SIRIUS_TASK))
with open('canopus_classifications.tsv', 'wb') as f:
    f.write(r.content)
canopus_df = pd.read_csv("canopus_classifications.tsv", sep="\t")

In [39]:
# Integrating into Graphml
G = nx.read_graphml("fbmn_network.graphml")

# Adding sirius information
for result in sirius_df.to_dict(orient="records"):
    scan = str(result["scan"])
    if scan in G:
        G.node[scan]["sirius:molecularFormula"] = result["molecularFormula"]
        G.node[scan]["sirius:adduct"] = result["adduct"]
        
# Adding fingerid information
for result in fingerid_df.to_dict(orient="records"):
    scan = str(result["scan"])
    if scan in G:
        G.node[scan]["csifingerid:smiles"] = result["smiles"]
        G.node[scan]["csifingerid:Confidence_Score"] = result["Confidence_Score"]
    
# Adding canopus information
for result in canopus_df.to_dict(orient="records"):
    scan = str(result["scan"])
    if scan in G:
        G.node[scan]["canopus:subclass"] = result["subclass"]
        G.node[scan]["canopus:class"] = result["class"]
        G.node[scan]["canopus:superclass"] = result["superclass"]
    
nx.write_graphml(G, "fbmn_sirius_network.graphml")